In [173]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mydsstocks3/dftokaggle/code_601010.csv


In [174]:
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler

In [175]:
def datapre_csv(spfile="/kaggle/input/mydsstocks3/dftokaggle/code_000001.csv"):
    df=None
    try:
        df=pd.read_csv(spfile)
#         print(df[:4])
        if df is not None and len(df)>1:
            df['date'] = pd.to_datetime(df['date'])
            #计算均线，要从小排序到大,ascending=True
            df=df.sort_values(by='date',ascending=True)
            for item in [5,10,20,30,50,60]:
                df['ma%s'%(item)]=df.close.rolling(item).mean()
            
            df=df[df.columns[1:]]
#             df.drop(labels='Unnamed'.split(","),inplace =True,axis=1)
            df=df.sort_values(by='date',ascending=False)
            for item in [5,10,20,30,50,60]:
                df['perma%s'%(item)]=(df['ma%s'%(item)]-df['ma%s'%(item)].shift(-1))/df['ma%s'%(item)].shift(-1)*100
            df = (df.replace([np.inf, -np.inf], np.nan)).fillna(0) 
            
    except Exception as e:
        print(e)
    return df

In [176]:
def datapre_lbl(df):
    df=df.copy()
    try:
        if df is not None and len(df)>1:
            df=df.sort_values(by='date',ascending=False)
            df["lbl"]=(df.close-df.close.shift(-1))/df.close.shift(-1)
            df["lbl1"]=df["lbl"].shift(1)##标签应该是第二天的变化幅度。
            
            bins = [-1.3,-0.05, -0.02, 0, 0.02,0.05, 0.07, 0.1,1.5]## 9 point
            labels = ['b1', 'b2', 'b3', 'g1','g2','g3','g4','g5']##8 area
            transformer = preprocessing.FunctionTransformer( pd.cut, 
                kw_args={'bins': bins, 'labels': labels, 'retbins': False}
            )
            df['lbl2']=transformer.fit_transform(df['lbl1'])
            lb = preprocessing.LabelEncoder()
            df['lbl3']=lb.fit_transform(df['lbl2'])
    except Exception as e:
        print(e)
    return df

In [177]:
def datapre_mm(df):
    df=df.copy()
    try:
        if df is not None and len(df)>1:
            colslist=['open', 'high', 'low', 'close', 'volume', 'outstanding_share',
                   'turnover', 'ma5', 'ma10', 'ma20', 'ma30', 'ma50', 'ma60']
            scaler = MinMaxScaler() #实例化
            df[colslist] = scaler.fit_transform(df[colslist]) #fit，在这里本质是生成min(x)和max(x)
            # df = (df.replace([np.inf, -np.inf], np.nan)).fillna(0) 
            # df=df.fillna(0
            df.drop(['lbl', 'lbl1', 'lbl2'], axis=1, inplace=True)
    except Exception as e:
        print(e)
    return df


In [178]:
spfile="/kaggle/input/mydsstocks3/dftokaggle/code_000001.csv"
code_=spfile.split("code_")[1].replace(".csv","")
df=datapre_csv(spfile=spfile)
df=datapre_lbl(df)
df=datapre_mm(df)
# df['close,date,lbl,lbl2,lbl3,lbl1'.split(",")][:11]
df[:6]

,date,open,high,low,close,volume,outstanding_share,turnover,ma5,ma10,...,ma30,ma50,ma60,perma5,perma10,perma20,perma30,perma50,perma60,lbl3
0,2023-09-07,0.554736,0.549437,0.578253,0.549000,0.079226,0.99788,0.011140,0.615795,0.617619,...,0.628015,0.651584,0.665828,0.021035,0.268152,0.019736,-0.112401,-0.089941,-0.001192,8
1,2023-09-06,0.559426,0.559203,0.582732,0.555827,0.133445,0.99788,0.028760,0.615666,0.615968,...,0.628721,0.652170,0.665836,0.125688,0.614060,-0.033871,-0.082252,-0.088088,-0.004938,2
2,2023-09-05,0.567802,0.564638,0.591323,0.562523,0.091711,0.99788,0.015197,0.614893,0.612208,...,0.629239,0.652745,0.665868,0.605588,0.608738,0.069887,-0.070501,-0.066755,0.048613,2
3,2023-09-04,0.570484,0.568081,0.591437,0.567453,0.127473,0.99788,0.026820,0.611192,0.608504,...,0.629683,0.653181,0.665545,0.727032,0.690480,0.030585,0.085247,0.002796,0.038747,2
4,2023-09-01,0.553711,0.564747,0.584019,0.564203,0.176547,0.99788,0.042768,0.606780,0.604331,...,0.629146,0.653163,0.665287,0.950792,0.567729,0.000874,0.047063,-0.012204,0.078066,3
5,2023-08-31,0.557070,0.556973,0.578024,0.548324,0.187268,0.99788,0.046252,0.601065,0.600920,...,0.628850,0.653243,0.664768,0.522595,0.121173,-0.131605,-0.059998,-0.037417,0.067023,4
6,2023-08-30,0.543002,0.550108,0.568951,0.551792,0.151981,0.99788,0.034785,0.597940,0.600193,...,0.629228,0.653487,0.664323,1.121913,0.288749,-0.132302,-0.076917,0.000341,0.084049,2
7,2023-08-29,0.541213,0.542443,0.567068,0.543198,0.140168,0.99788,0.030945,0.591307,0.598464,...,0.629712,0.653485,0.663765,0.612014,0.161819,-0.196064,-0.040448,-0.080222,0.124177,3
8,2023-08-28,0.557070,0.558987,0.569778,0.544419,0.192628,0.99788,0.047994,0.587710,0.597498,...,0.629967,0.654010,0.662942,0.652496,0.131117,-0.280866,-0.056916,-0.064178,0.212831,2
9,2023-08-25,0.530809,0.536899,0.554963,0.534364,0.100416,0.99788,0.018026,0.583900,0.596715,...,0.630326,0.654430,0.661534,0.172723,0.014571,-0.396560,-0.128897,-0.063389,0.187126,3


In [180]:
spathsave="/kaggle/working/datapre1/"
if not os.path.exists(spathsave):os.makedirs(spathsave)
df.to_csv(os.path.join(spathsave, "%s_pre%s"%(code_,".csv")))

In [183]:
for dirname, _, filenames in os.walk(spathsave):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

/kaggle/working/datapre1/000001_pre.csv


In [186]:
# /kaggle/working/datapre1
import glob
from tqdm import tqdm

listfile=glob.glob("/kaggle/input/mydsstocks3/dftokaggle/*.csv")
spathsave="/kaggle/working/datapre1/"
if not os.path.exists(spathsave):os.makedirs(spathsave)
        
# print(listfile)
for item in tqdm(listfile):
    spfile=item
    code_=spfile.split("code_")[1].replace(".csv","")
    df=datapre_csv(spfile=spfile)
    df=datapre_lbl(df)
    df=datapre_mm(df)
    
    df.to_csv(os.path.join(spathsave, "%s_pre%s"%(code_,".csv")))

100%|██████████| 5347/5347 [14:39<00:00,  6.08it/s]
